In [1]:
import sqlite3
import threading
from flask import Flask, render_template, request, redirect, session,url_for

app = Flask(__name__)

app.secret_key = 'LOkeHarSHa@1903'

# Create a thread-local storage for SQLite connections
thread_local = threading.local()

def get_db():
    # Get the SQLite connection for the current thread
    if not hasattr(thread_local, 'connection'):
        thread_local.connection = sqlite3.connect('customer.db')
    return thread_local.connection

def get_employees_db():
    # Get the SQLite connection for the employees database
    if not hasattr(thread_local, 'employees_connection'):
        thread_local.employees_connection = sqlite3.connect('employee.db')
    return thread_local.employees_connection

def close_db(exception=None):
    # Close the SQLite connections for the current thread
    if hasattr(thread_local, 'connection'):
        thread_local.connection.close()
    if hasattr(thread_local, 'employees_connection'):
        thread_local.employees_connection.close()


@app.route('/')
def index():
    image_paths = ['/static/1.jpeg', '/static/2.jpeg', '/static/3.jpeg','/static/4.jpeg', '/static/6.jpeg', '/static/7.jpeg','/static/8.jpeg', '/static/9.jpeg', '/static/bank.jpeg']  # list of image paths
    return render_template('index.html', image_paths=image_paths)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        # Perform authentication against the employee database
        employees_db = get_employees_db()
        employees_cursor = employees_db.cursor()
        employees_cursor.execute("SELECT * FROM employees WHERE username = ? AND password = ?", (username, password))
        employee = employees_cursor.fetchone()

        if employee:
            # Authentication successful
            session['username'] = username
            return redirect('/dashboard')
        else:
            # Authentication failed
            return render_template('login.html', error='Invalid credentials')

    return render_template('login.html')

@app.route('/dashboard')
def dashboard():
    # Check if the user is authenticated
    if 'username' not in session:
        return redirect('/login')

    return render_template('dashboard.html')
@app.route('/customers', methods=['GET', 'POST'])
def customers():
    
    db = get_db()
    cursor = db.cursor()
    if request.method == 'POST':
        cust_id = request.form['cust_id']
        name = request.form['name']
        city = request.form['city']
        gender = request.form['gender']
        dob = request.form['dob']
        occupation = request.form['occupation']
        
        cursor.execute('''
            INSERT INTO customers (cust_id, cust_name, city, gender, dob, occupation)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (cust_id, name, city, gender, dob, occupation))
        db.commit()
        return redirect('/customers')
    cursor.execute('SELECT * FROM customers')
    rows = cursor.fetchall()
    cursor.close()

    return render_template('customers.html', rows=rows)

@app.route('/loans', methods=['GET', 'POST'])
def loans():
    db = get_db()
    cursor = db.cursor()
    if request.method == 'POST':
        loan_no = request.form['loan_no']
        cust_id = request.form['cust_id']
        branch_id = request.form['branch_id']
        loan_amount = request.form['loan_amount']
        
        cursor.execute('''
            INSERT INTO loans (loan_no, cust_id, branch_id, loan_amount)
            VALUES (?, ?, ?, ?)
        ''', (loan_no, cust_id, branch_id, loan_amount))
        db.commit()
        return redirect('/loans')
    cursor.execute('SELECT * FROM loans')
    rows = cursor.fetchall()
    cursor.close()

    return render_template('loans.html', rows=rows)

@app.route('/branches', methods=['GET', 'POST'])
def branches():
    db = get_db()
    cursor = db.cursor()
    if request.method == 'POST':
        branch_id = request.form['branch_id']
        branch_name = request.form['branch_name']
        city = request.form['city']
        assets = request.form['assets']
        
        cursor.execute('''
            INSERT INTO branches (branch_id, branch_name, city, assets)
            VALUES (?, ?, ?, ?)
        ''', (branch_id, branch_name, city, assets))
        db.commit()
        return redirect('/branches')
    cursor.execute('SELECT * FROM branches')
    rows = cursor.fetchall()
    cursor.close()

    return render_template('branches.html', rows=rows)

@app.route('/accounts', methods=['GET', 'POST'])
def accounts():
    db = get_db()
    cursor = db.cursor()
    if request.method == 'POST':
        acc_no = request.form['acc_no']
        cust_id = request.form['cust_id']
        branch_id = request.form['branch_id']
        balance = request.form['balance']
        acc_type = request.form['acc_type']
        status = request.form['status']
        
        cursor.execute('''
            INSERT INTO accounts (acc_no, cust_id, branch_id, balance, acc_type, status)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (acc_no, cust_id, branch_id, balance, acc_type, status))
        db.commit()
        return redirect('/accounts')
    cursor.execute('SELECT * FROM accounts')
    rows = cursor.fetchall()
    cursor.close()

    return render_template('accounts.html', rows=rows)

@app.route('/transactions', methods=['GET', 'POST'])
def transactions():
    db = get_db()
    cursor = db.cursor()
    if request.method == 'POST':
        trans_id = request.form['trans_id']
        acc_no = request.form['acc_no']
        amount = request.form['amount']

        # Check if 'trans_mode' key exists in the form data
        if 'transaction_mode' in request.form:
            trans_mode = request.form['transaction_mode']
        else:
            trans_mode = None

        # Check if 'trans_date' key exists in the form data
        if 'trans_date' in request.form:
            trans_date = request.form['trans_date']
        else:
            trans_date = None

        cursor.execute('''
            INSERT INTO transactions (trans_id, acc_no, trans_date, transaction_mode, amount)
            VALUES (?, ?, ?, ?, ?)
        ''', (trans_id, acc_no, trans_date, trans_mode, amount))

        cursor.execute('''
            UPDATE accounts
            SET balance = balance + ?
            WHERE acc_no = ?
        ''', (amount, acc_no))
        db.commit()
        return redirect('/transactions')

    cursor.execute('SELECT * FROM transactions')
    rows = cursor.fetchall()

    cursor.close()

    return render_template('transactions.html', rows=rows)

@app.route('/logout', methods=['GET','POST'])
def logout():
    # Perform logout operations
    # For example, clear the session or perform any necessary cleanup

    # Redirect to the index page
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.teardown_appcontext(close_db)
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/bank.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/2.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/1.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/3.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/4.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/6.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/7.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/9.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /static/8.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [14/Jul/2023 14:23:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Jul/2023 14:23:47] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [14/Jul/2023 14:23:47] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [14